In [1]:
try:
    import cvxopt
except:
    !pip install cvxopt
    import cvxopt

In [2]:
%matplotlib notebook
import numpy as np
# import cvxopt
import cvxopt.glpk

mainsize=3

In [3]:
## help for the integer LP solver in cvxopt
cvxopt.glpk.ilp?

In [4]:
## To help create the huge matrix


def unravel(i,j,k,size=mainsize):
    '''
    Associate a unique variable index given a 3-index (ijk) 
    '''
    n2 = size*size
    assert(i>=0 and i<n2)
    assert(j>=0 and i<n2)
    assert(k>=0 and i<n2)
    
    return(k+ j*n2+ i*n2*n2)


""" not used, for reference only
def ravel(l,size=3):
    n2=size*size
    assert (l>=0 and l < n2*n2*n2)
    i = l // (n2*n2)
    j = (l % (n2*n2)) // n2
    k = l - i*n2*n2 - j*n2
    return ((i,j,k))
"""

' not used, for reference only\ndef ravel(l,size=3):\n    n2=size*size\n    assert (l>=0 and l < n2*n2*n2)\n    i = l // (n2*n2)\n    j = (l % (n2*n2)) // n2\n    k = l - i*n2*n2 - j*n2\n    return ((i,j,k))\n'

In [5]:
## create constraint matrix

size=3
n2=size*size
A=np.zeros((4*n2*n2,n2*n2*n2))
A.shape

(324, 729)

In [6]:
## Here I give the line constraints
## line constraints: only one number per line
c=0
for k in range(n2): ## for all numbers
    for j in range(n2): ## for all columns
        for i in range(n2):
            A[c,unravel(i,j,k)] = 1 ## only one number k on line i
#             if c == 9:
#                 print(unravel(i,j,k))
        c += 1
    
## column constraints: only one number per column
for k in range(n2): ## for all numbers
    for i in range(n2): ## for all lines
        for j in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number k on column j
        c += 1

## square constraints: only one number per square
for k in range(n2):
    for q in range(1,size+1): ## for all lines
        for p in range(1, size+1):
            for j in range((size*q-size+1) -1, (size*q)):
                for i in range((size*p-size+1)-1, (size*p)):
                    A[c,unravel(i,j,k)] = 1 ## only one number k on column j
            c += 1
## Assignment constraint
for i in range(n2):
    for j in range(n2):
        for k in range(n2):
            A[c,unravel(i,j,k)] = 1
        c += 1

## You need to write the remaining constraints (columns, subsquares, etc)

      
        
print("Total number of constraints=",c)

def testA(A,c,size=3):
    n2 = size*size
    for n in range(c):
        if (np.sum(A[n,])!=n2):
            print("error on line", n)
            break
    print("All constraints OK")
    return

testA(A,c)

162
243
Total number of constraints= 324
All constraints OK


In [7]:
## print solution
def printsol(sol):
    sep3="+-----+-----+-----+"
    for i in range(n2):
        if (i%3 == 0):
            print(sep3)
        print("|",end='')
        for j in range(n2):
            for k in range(n2):
                if (sol[unravel(i,j,k)]==1):
                    print(k+1,end='')
            if (j%3 ==2):
                print("|",end='')
            else:
                print(" ",end='')
        print("")
    print(sep3)

In [10]:
## here are the given number constraints
## add the fixed numbers constraints
from cvxopt import matrix
def solve_sudoku(K, A):
    for i in range(n2):
        for j in range(n2):
            k = K[i,j]
            if (k>0):
                newrow=np.zeros(n2*n2*n2)
                newrow[unravel(i,j,k-1)]=1
                A=np.vstack((A,newrow))
    b=matrix(np.ones(A.shape[0])) ## set partition
    c=matrix(np.zeros(A.shape[1])) ## zero cost
    G=matrix(np.zeros(A.shape))
    h=matrix(np.zeros(A.shape[0]))
    binary=np.array(range(A.shape[1]))
    I=set(binary)
    B=set(range(A.shape[1]))
    Aeq = matrix(A)
    (status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))
    print("The status of the solution is:", status)
    printsol(solution)
    

In [11]:
## here are the given number constraints
## add the fixed numbers constraints
K = np.array([
[0,8,0, 9,0,1, 0,5,0],
[0,0,2, 6,8,7, 3,0,0],
[0,0,3, 0,0,0, 6,0,0],
[3,9,0, 0,0,0, 0,6,5],
[6,0,0, 4,7,5, 0,0,3],
[5,7,0, 0,0,0, 0,8,4],
[0,0,9, 0,0,0, 8,0,0],
[0,0,5, 1,2,4, 9,0,0],
[0,4,0, 8,0,3, 0,2,0]])
solve_sudoku(K, A)

The status of the solution is: optimal
+-----+-----+-----+
|7 8 6|9 3 1|4 5 2|
|4 5 2|6 8 7|3 1 9|
|9 1 3|5 4 2|6 7 8|
+-----+-----+-----+
|3 9 4|2 1 8|7 6 5|
|6 2 8|4 7 5|1 9 3|
|5 7 1|3 6 9|2 8 4|
+-----+-----+-----+
|2 3 9|7 5 6|8 4 1|
|8 6 5|1 2 4|9 3 7|
|1 4 7|8 9 3|5 2 6|
+-----+-----+-----+


In [13]:
K = np.array([
[7,0,0, 0,0,0, 4,0,0],
[0,2,0, 0,7,0, 0,8,0],
[0,0,3, 0,0,8, 0,0,9],
[0,0,0, 5,0,0, 3,0,0],
[0,6,0, 0,2,0, 0,9,0],
[0,0,1, 0,0,7, 0,0,6],
[0,0,0, 3,0,0, 9,0,0],
[0,3,0, 0,4,0, 0,6,0],
[0,0,9, 0,0,1, 0,0,5]])
solve_sudoku(K, A)

The status of the solution is: optimal
+-----+-----+-----+
|7 9 8|6 3 5|4 2 1|
|1 2 6|9 7 4|5 8 3|
|4 5 3|2 1 8|6 7 9|
+-----+-----+-----+
|9 7 2|5 8 6|3 1 4|
|5 6 4|1 2 3|8 9 7|
|3 8 1|4 9 7|2 5 6|
+-----+-----+-----+
|6 1 7|3 5 2|9 4 8|
|8 3 5|7 4 9|1 6 2|
|2 4 9|8 6 1|7 3 5|
+-----+-----+-----+
